# Teste de Nivelamento Intuitive Care

## Teste Web Scraping

In [ ]:
## Este programa executa o download dos arquivos .pdf solicitados e os organiza dentro de uma pasta compactada no formato .zip
## Caio Sousa Ferreira

import requests
import zipfile
import os

print('Download dos Arquivos\n')
print('Iniciando\n')


# Links para os arquivos necessários
url_Arquivo1 = 'https://www.gov.br/ans/pt-br/acesso-a-informacao/participacao-da-sociedade/atualizacao-do-rol-de-procedimentos/Anexo_I_Rol_2021RN_465.2021_RN627L.2024.pdf'
url_Arquivo2 = 'https://www.gov.br/ans/pt-br/acesso-a-informacao/participacao-da-sociedade/atualizacao-do-rol-de-procedimentos/Anexo_II_DUT_2021_RN_465.2021_RN628.2025_RN629.2025.pdf'

# Solicitação HTTP dos arquivos
conteudo_Arquivo1 = requests.get(url_Arquivo1)
#nome_Arquivo1 = 'AnexoI.pdf'

conteudo_Arquivo2 = requests.get(url_Arquivo2)
#nome_Arquivo2 = 'AnexoII.pdf'

# Baixar e gerar os arquivos na pasta (Anexo I)
if conteudo_Arquivo1.status_code == 200:
    with open('Anexo I.pdf', 'wb') as file:
        file.write(conteudo_Arquivo1.content)
    print('Arquivo 1 baixado com sucesso')
else:
    print('Não foi possível baixar o arquivo')

    
# Baixar e gerar os arquivos na pasta (Anexo II)
if conteudo_Arquivo2.status_code == 200:
    with open('Anexo II.pdf', 'wb') as file:
        file.write(conteudo_Arquivo2.content)
    print('Arquivo 2 baixado com sucesso')
else:
    print('Não foi possível baixar o arquivo')

# Criar pasta .zip para compactar os arquivos
with zipfile.ZipFile('arquivos_anexos.zip', 'w') as myzip:
    myzip.write('Anexo I.pdf')
    myzip.write('Anexo II.pdf')

#os.remove('Anexo I.pdf')
os.remove('Anexo II.pdf')

input('\nAperte qualquer tecla para sair')





## Teste transformação de dados

In [ ]:
## Este código faz a leitura do arquivo, altera as colunas solicitadas e gera o arquivo final .csv dentro de uma pasta compactada no formato
## .zip com o nome solicitado

import tabula
import pandas as pd
import os
import zipfile

# Carregamento de todas as tabelas a partir do .pdf
lista_tabelas = tabula.read_pdf('Anexo I.pdf', pages="3-181")

# Alteração das colunas em todas as tabelas
for df in lista_tabelas:

    df.rename(columns = {'OD':'Seg. Odontológica'}, inplace=True)
    df.rename(columns = {'AMB':'Seg. Ambulatorial'}, inplace=True)

# Juntar e criar o arquivo .csv com todas as tabelas
for df in lista_tabelas:
    
    df.to_csv('Teste_{Caio_Ferreira}.csv', index=False, encoding="utf-8-sig", sep=';', mode='a')

# Compactar o arquivo em uma pasta .zip e remover a cópia do arquivo do diretório
with zipfile.ZipFile('Teste_{Caio_Ferreira}.zip', 'w') as myzip:
    myzip.write('Teste_{Caio_Ferreira}.csv')
os.remove('Teste_{Caio_Ferreira}.csv')

## Teste de Banco de Dados

In [29]:
import pandas
import pandasql as ps
pandas.options.display.max_rows = 9999

In [30]:
# Importando todos os arquivos .csv
tri1_2023 = pandas.read_csv('1T2023.csv', delimiter=';')
tri2_2023 = pandas.read_csv('2T2023.csv', delimiter=';')
tri3_2023 = pandas.read_csv('3T2023.csv', delimiter=';')
tri4_2023 = pandas.read_csv('4T2023.csv', delimiter=';')
tri1_2024 = pandas.read_csv('1T2024.csv', delimiter=';')
tri2_2024 = pandas.read_csv('2T2024.csv', delimiter=';')
tri3_2024 = pandas.read_csv('3T2024.csv', delimiter=';')
tri4_2024 = pandas.read_csv('4T2024.csv', delimiter=';')

lista_operadoras = pandas.read_csv('Relatorio_cadop.csv', delimiter=';')

In [31]:
# Conjunto dos arquivos que formam o último ano
ultimo_ano = [tri4_2024, tri3_2024, tri2_2024, tri1_2024]

# Gerando arquivo .csv para consulta posterior
for df in ultimo_ano:
    df.to_csv('ultimo_ano.csv', index=False, sep=';', encoding='utf-8-sig', mode='a')

ultimo_ano_csv = pandas.read_csv('ultimo_ano.csv', delimiter=';')

C:\Users\caios\AppData\Local\Temp\ipykernel_22020\929362586.py:8: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  ultimo_ano_csv = pandas.read_csv('ultimo_ano.csv', delimiter=';')


In [32]:
# Query para selecionar: Quais as 10 operadoras com maiores despesas em "EVENTOS/ SINISTROS CONHECIDOS OU
# AVISADOS DE ASSISTÊNCIA A SAÚDE MEDICO HOSPITALAR" no último trimestre?

query1 = """
SELECT
    tri4_2024.DATA,
    tri4_2024.REG_ANS,
    lista_operadoras.Razao_Social,
    tri4_2024.DESCRICAO,
    tri4_2024.VL_SALDO_INICIAL,
    tri4_2024.VL_SALDO_FINAL
FROM tri4_2024
JOIN lista_operadoras
ON tri4_2024.REG_ANS = lista_operadoras.Registro_ANS
WHERE DESCRICAO = 'EVENTOS/ SINISTROS CONHECIDOS OU AVISADOS  DE ASSISTÊNCIA A SAÚDE MEDICO HOSPITALAR '
ORDER BY CAST (VL_SALDO_FINAL AS SIGNED) ASC
LIMIT 10;
"""

resultado1 = ps.sqldf(query1, locals())

#### Quais as 10 operadoras com maiores despesas em "EVENTOS/ SINISTROS CONHECIDOS OU AVISADOS DE ASSISTÊNCIA A SAÚDE MEDICO HOSPITALAR" no último trimestre?

In [33]:
# Apresentando o resultado para verificação
resultado1

,DATA,REG_ANS,Razao_Social,DESCRICAO,VL_SALDO_INICIAL,VL_SALDO_FINAL
0,2024-10-01,359017,NOTRE DAME INTERMÉDICA SAÚDE S.A.,EVENTOS/ SINISTROS CONHECIDOS OU AVISADOS DE ...,"23591364,89","-181349151,13"
1,2024-10-01,368253,HAPVIDA ASSISTENCIA MEDICA S.A.,EVENTOS/ SINISTROS CONHECIDOS OU AVISADOS DE ...,"116198901,2","-127840720,03"
2,2024-10-01,340782,CLINIPAM CLINICA PARANAENSE DE ASSISTENCIA MED...,EVENTOS/ SINISTROS CONHECIDOS OU AVISADOS DE ...,"6977606,44","-13784896,92"
3,2024-10-01,393533,ASSOCIAÇÃO DO FISCO DE ALAGOAS,EVENTOS/ SINISTROS CONHECIDOS OU AVISADOS DE ...,"-4695266,13","-7711963,73"
4,2024-10-01,392804,CENTRO CLÍNICO GAÚCHO LTDA,EVENTOS/ SINISTROS CONHECIDOS OU AVISADOS DE ...,"1797584,51","-4482762,91"
5,2024-10-01,363286,UNIMED DE TAUBATÉ COOPERATIVA DE TRABALHO MÉDICO,EVENTOS/ SINISTROS CONHECIDOS OU AVISADOS DE ...,"-667693,37","-680447,85"
6,2024-10-01,416428,SUL AMÉRICA SERVIÇOS DE SAÚDE S.A.,EVENTOS/ SINISTROS CONHECIDOS OU AVISADOS DE ...,"111629,38","-656879,78"
7,2024-10-01,350249,H.B. SAÚDE S/A.,EVENTOS/ SINISTROS CONHECIDOS OU AVISADOS DE ...,"1548069,97","-624561,68"
8,2024-10-01,477,SUL AMÉRICA SEGURADORA DE SAÚDE S.A.,EVENTOS/ SINISTROS CONHECIDOS OU AVISADOS DE ...,"50581,17","-372237,4"
9,2024-10-01,329355,IRMANDADE DE MISERICORDIA DE PORTO FERREIRA,EVENTOS/ SINISTROS CONHECIDOS OU AVISADOS DE ...,"-316181,03","-310072,62"


In [38]:
# Gerando arquivo .csv com o resultado da primeira consulta
resultado1.to_csv('resultado1.csv', index=False, sep=';', encoding='utf-8-sig')

In [39]:
# Query para selecionar: Quais as 10 operadoras com maiores despesas em "EVENTOS/ SINISTROS CONHECIDOS OU
# AVISADOS DE ASSISTÊNCIA A SAÚDE MEDICO HOSPITALAR" no último ano?

query2 = """
SELECT
    ultimo_ano_csv.DATA,
    ultimo_ano_csv.REG_ANS,
    lista_operadoras.Razao_Social,
    ultimo_ano_csv.DESCRICAO,
    ultimo_ano_csv.VL_SALDO_INICIAL,
    ultimo_ano_csv.VL_SALDO_FINAL
FROM ultimo_ano_csv
JOIN lista_operadoras
ON ultimo_ano_csv.REG_ANS = lista_operadoras.Registro_ANS
WHERE DESCRICAO = 'EVENTOS/ SINISTROS CONHECIDOS OU AVISADOS  DE ASSISTÊNCIA A SAÚDE MEDICO HOSPITALAR '
ORDER BY CAST (VL_SALDO_FINAL AS SIGNED) ASC
LIMIT 10;
"""

resultado2 = ps.sqldf(query2, locals())

#### Quais as 10 operadoras com maiores despesas em "EVENTOS/ SINISTROS CONHECIDOS OU AVISADOS DE ASSISTÊNCIA A SAÚDE MEDICO HOSPITALAR" no último ano?

In [40]:
# Apresentando o resultado para verificação
resultado2

,DATA,REG_ANS,Razao_Social,DESCRICAO,VL_SALDO_INICIAL,VL_SALDO_FINAL
0,2024-10-01,359017,NOTRE DAME INTERMÉDICA SAÚDE S.A.,EVENTOS/ SINISTROS CONHECIDOS OU AVISADOS DE ...,"23591364,89","-181349151,13"
1,2024-10-01,359017,NOTRE DAME INTERMÉDICA SAÚDE S.A.,EVENTOS/ SINISTROS CONHECIDOS OU AVISADOS DE ...,"23591364,89","-181349151,13"
2,2024-10-01,368253,HAPVIDA ASSISTENCIA MEDICA S.A.,EVENTOS/ SINISTROS CONHECIDOS OU AVISADOS DE ...,"116198901,2","-127840720,03"
3,2024-10-01,368253,HAPVIDA ASSISTENCIA MEDICA S.A.,EVENTOS/ SINISTROS CONHECIDOS OU AVISADOS DE ...,"116198901,2","-127840720,03"
4,2024-10-01,340782,CLINIPAM CLINICA PARANAENSE DE ASSISTENCIA MED...,EVENTOS/ SINISTROS CONHECIDOS OU AVISADOS DE ...,"6977606,44","-13784896,92"
5,2024-10-01,340782,CLINIPAM CLINICA PARANAENSE DE ASSISTENCIA MED...,EVENTOS/ SINISTROS CONHECIDOS OU AVISADOS DE ...,"6977606,44","-13784896,92"
6,2024-10-01,393533,ASSOCIAÇÃO DO FISCO DE ALAGOAS,EVENTOS/ SINISTROS CONHECIDOS OU AVISADOS DE ...,"-4695266,13","-7711963,73"
7,2024-10-01,393533,ASSOCIAÇÃO DO FISCO DE ALAGOAS,EVENTOS/ SINISTROS CONHECIDOS OU AVISADOS DE ...,"-4695266,13","-7711963,73"
8,2024-07-01,393533,ASSOCIAÇÃO DO FISCO DE ALAGOAS,EVENTOS/ SINISTROS CONHECIDOS OU AVISADOS DE ...,"-4173038,16","-4695266,13"
9,2024-07-01,393533,ASSOCIAÇÃO DO FISCO DE ALAGOAS,EVENTOS/ SINISTROS CONHECIDOS OU AVISADOS DE ...,"-4173038,16","-4695266,13"


In [41]:
# Gerando arquivo .csv com o resultado da primeira consulta
resultado2.to_csv('resultado2.csv', index=False, sep=';', encoding='utf-8-sig')